In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
import warnings

from data_describe.text.text_preprocessing import *
from sklearn.datasets import fetch_20newsgroups

warnings.simplefilter('ignore')

In [ ]:
import sys
sys.path.append("../scripts/")
from data_cleaner import DataCleaner
from util import Util
from preprocessor import Processor

In [ ]:
util = Util()
cleaner = DataCleaner()
processor = Processor()

# News

In [ ]:
repo="https://github.com/Nathnael12/prompt-engineering.git"
news_df=util.read_from_dvc("data/news.csv",repo,"news-v0",low_memory=False)

# cleaned_df=util.read_from_file('../data/news.csv')

In [ ]:
news_df=news_df.sample(frac=1)

train_news=news_df.head(8)
test_news=news_df.tail(2)

In [ ]:
full_processed_df=processor.prepare_text(train_news.copy())
description_processed=processor.prepare_text(train_news.copy(),columns=["Description"])[["Description","Analyst_Average_Score"]]
title_processed=processor.prepare_text(train_news.copy(),columns=["Title"])[["Title","Analyst_Average_Score"]]
body_processed=processor.prepare_text(train_news.copy(),columns=["Body"])[["Body","Analyst_Average_Score"]]
unprocessed=train_news

processed_test=processor.prepare_text(test_news)

In [ ]:
full_processed_df['Description']=full_processed_df['Title'] + ", " + full_processed_df['Description'] + ", " + full_processed_df["Body"]
full_processed_df.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

title_processed.rename(columns={'Title':'Description'},inplace=True)

body_processed.rename(columns={'Body':'Description'},inplace=True)

unprocessed['Description']=unprocessed['Title'] + ", " + unprocessed['Description'] + ", " + unprocessed["Body"]
unprocessed.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)

processed_test['Description']=processed_test['Title'] + ", " + processed_test['Description'] + ", " + processed_test["Body"]
processed_test.drop(['Domain','Title','Body','Link','timestamp','Analyst_Rank','Reference_Final_Score'],axis=1,inplace=True)




In [ ]:
frames = [full_processed_df, description_processed, title_processed, body_processed, unprocessed]

full_promp_trainer = pd.concat(frames)


In [ ]:
full_promp_trainer=full_promp_trainer.reset_index().drop(['index'],axis=1)

In [ ]:
# full_promp_trainer=full_promp_trainer[['Domain','Title','Description','Body','timestamp','Analyst_Average_Score']]
full_promp_trainer.rename(columns={"Analyst_Average_Score":"Analyst Average Score"},inplace=True)
# full_promp_trainer=cleaner.convert_to_datetime(full_promp_trainer,['timestamp'])
# full_promp_trainer=full_promp_trainer.fillna('')
full_promp_trainer.head()

In [ ]:

processor.prepare_tuner(full_promp_trainer)

In [ ]:
# uncomment these for first time use
# full_processed_df.to_csv("../data/trainer_news.csv",index=False)
# processed_test.to_csv("../data/test_news.csv",index=False)
# full_processed_df.to_csv("../data/processed_news.csv",index=False)


# Job

In [ ]:
job_df=pd.read_json("../data/job_description_train.json")
test_df=pd.read_json("../data/job_description_test.json")

In [ ]:
job_df_train=job_df.copy()
job_df_test = test_df.copy()

processed_description=processor.prepare_job_description_text(job_df_train)
processed_test_description=processor.prepare_job_description_text(job_df_test)

display(processed_description.head())
display(processed_test_description.head())

In [ ]:
processed_test_description.shape

In [ ]:
processed_description.shape

In [ ]:
job_frames = [processed_description, processed_test_description.head(10)]

job_tuner_df = pd.concat(job_frames)

In [ ]:
job_tuner_df=job_tuner_df.reset_index().drop(["index"],axis=1)

In [ ]:
processor.prepare_job_description_tuner(job_tuner_df)